In [ ]:
! pip install kagglehub

In [ ]:
import kagglehub

from kagglehub import KaggleDatasetAdapter

In [ ]:
file_path = "coin_Bitcoin.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "sudalairajkumar/cryptocurrencypricehistory",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)